In [1]:
# ! wget 'https://raw.githubusercontent.com/shyanne399/604_project/master/weight_diff.txt'

In [2]:
import numpy as np
import time
import pandas as pd
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [3]:
def puf_query(c, w):
  n = c.shape[1]
  phi = np.ones(n+1)
  phi[n] = 1
  for i in range(n-1, -1, -1):
    phi[i] = (2*c[0,i]-1)*phi[i+1]
    r = (np.dot(phi, w) > 0)
  return r

In [4]:
target = 0.99 # The desired prediction rate
n = 64 # number of stages in the PUF

# Initialize the PUF
np.random.seed(int(time.time()))
data = np.loadtxt('./weight_diff.txt')
w = np.zeros((n+1, 1))
for i in range(1, n+2):
  randi_offset = np.random.randint(1, 45481)
  w[i-1] = data[randi_offset-1]
  # print(len(data),data,data[randi_offset-1],randi_offset-1,w[i-1],w)

In [5]:
def calc_phi(select_bits):
    phi_vals = []
    for i in range(len(select_bits)):
      target_slice = select_bits[i:]
      zeros = [z for z in target_slice if z == 0]
      phi = 1 if len(zeros) % 2 == 0 else -1
      phi_vals.append(phi)
    return np.array(phi_vals + [1])

# gridsearchcv

In [6]:
# # gridsearchcv
# training_size = [1000,3000,5000,8000,10000,12000]  # Number of training samples
# X_train_list, y_train_list = [],[]

# for items in training_size:
#   X_train = np.random.randint(0, 2, size=(items, n))  # Random challenge vectors
#   y_train = np.zeros((items, 1))  # Response bits
#   for i in range(items):
#     y_train[i] = puf_query(X_train[i].reshape(1, -1), w)
#   X_train_list.append(X_train)
#   y_train_list.append(y_train)

In [7]:
# # gridsearchcv
# training_size = [5000]  # Number of training samples
# X_train_list, y_train_list = [],[]

# for items in training_size:
#   X_train = np.random.randint(0, 2, size=(items, n))  # Random challenge vectors
#   y_train = np.zeros((items, 1))  # Response bits
#   for i in range(items):
#     y_train[i] = puf_query(X_train[i].reshape(1, -1), w)
#   X_train_list.append(X_train)
#   y_train_list.append(y_train)

# c_range = np.arange(95,110,1)
# gamma_range = np.arange(0.1,0.5,.1)
# param_grid = {'C':c_range, 'gamma' : gamma_range}

# for X_raw,y_train in zip(X_train_list,y_train_list):
#   X_train_phi = np.apply_along_axis(calc_phi, 1, X_raw)
#   X = np.array(X_train_phi)
#   y = np.array([-1 if _y == 0 else 1 for _y in y_train])
#   SVC = svm.SVC()
#   grid = GridSearchCV(SVC,param_grid,refit=True,verbose=2)
#   grid.fit(X, y)

# print(grid.best_estimator_, len(X_raw),len(y_train))

# okay

In [8]:
# Evaluate your result

training_size = 5000 # Number of training samples

X_train = np.random.randint(0, 2, size=(training_size, n))  # Random challenge vectors
y_train = np.zeros((training_size, 1))  # Response bits
for i in range(training_size):
  y_train[i] = puf_query(X_train[i].reshape(1, -1), w)

X_train_phi = np.apply_along_axis(calc_phi, 1, X_train)

t0 = time.process_time()
X = np.array(X_train_phi)
y = np.array([-1 if _y == 0 else 1 for _y in y_train])
# c_range = [115,116,117]
# gamma_range = [0.01,0.02,0.03]

# for i in range(3):
#   SVC = svm.SVC(C=c_range[i], gamma=gamma_range[i], kernel = 'linear')
#   SVC.fit(X, y)

SVC = svm.SVC(C=120, gamma=0.02, kernel = 'linear')
SVC.fit(X, y)

t1 = time.process_time()

training_time = t1 - t0
print("Training time:", training_time)
print("Training size:", training_size)

correct = 0
w0 = SVC.coef_[0]

n_test = 10000
for i in range(1, n_test+1):
  c_test = np.random.randint(0, 2, size=(1, n)) # a random challenge vector
  r = puf_query(c_test, w)
  r0 = puf_query(c_test, w0)
  correct += (r==r0)
success_rate = correct/n_test
print("Success rate:", success_rate)


# If the success rate is less than 99%, a penalty time will be added
# One second is add for each 0.01% below 99%.
effective_training_time = training_time
if success_rate < 0.99:
  effective_training_time = training_time + 10000*(0.99-success_rate)
print("Effective training time:", effective_training_time)

Training time: 0.859375
Training size: 5000
Success rate: [0.9856]
Effective training time: [44.859375]
